In [1]:
import pandas as pd

from keras.models import Sequential
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation
from keras.layers.core import Dropout
from keras.layers.core import Dense,Reshape
from keras.layers import Flatten
from keras.layers import Input
from keras.models import Model

from sklearn.model_selection import train_test_split
from keras.layers.core import Dense
from keras.models import Model
from keras.optimizers import Adam
from keras.layers import concatenate
import numpy as np
import argparse
import locale
import os
import datetime

/Users/Schlendrikovic/Documents/Python/VirtualEnvironments/DeepLearning/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


---

# Data

In [2]:
df = pd.read_csv("data/data_preprocessed.csv",sep=";",index_col=[0])

df_prev_day_info = df.loc[:,'prev_day_consumption_ger':'12']
df_prev_day_info['Tag'] = df['Tag']

df['Tag'] = pd.to_datetime(df['Tag'])
df_prev_day_info['Tag'] = pd.to_datetime(df_prev_day_info['Tag'])


In [3]:
df_master = pd.DataFrame(index=np.arange(0,df.Tag.nunique()))
df_master['Tag'] = None

df_master['NX_per_country'] = None

In [4]:
mean_columns = df.loc[:,'NX':'PL'].mean()
std_columns = df.loc[:,'NX':'PL'].std()

max_columns =df.loc[:,'NX':'PL'].max()
min_columns =df.loc[:,'NX':'PL'].min()

#df.loc[:,'NX':'PL'] = (df.loc[:,'NX':'PL']-mean_columns)/std_columns

In [5]:
df = df.fillna(0)

In [6]:
ind = 0
for tag,group in df.groupby('Tag'):
    if len(group)==24:
        df_master.loc[ind,'Tag'] =  tag
        df_master.loc[ind,'NX_per_country'] =  group.loc[:,'NX':'PL'].values
    else:
        df_master.loc[ind,'NX_per_country'] = df_master.loc[ind-1,'NX_per_country'] 
        df_master.loc[ind,'Tag'] = df_master.loc[ind-1,'Tag'] 
        print("Tag weniger als 24 EInträge:{} {}".format(tag,len((group))))
    ind +=1

Tag weniger als 24 EInträge:2015-10-25 00:00:00 25
Tag weniger als 24 EInträge:2016-03-27 00:00:00 23
Tag weniger als 24 EInträge:2016-10-30 00:00:00 25
Tag weniger als 24 EInträge:2017-03-26 00:00:00 23
Tag weniger als 24 EInträge:2017-10-29 00:00:00 25
Tag weniger als 24 EInträge:2018-03-25 00:00:00 23
Tag weniger als 24 EInträge:2018-10-28 00:00:00 25
Tag weniger als 24 EInträge:2019-03-31 00:00:00 23


In [7]:
df_master['NX_per_country_prev_day'] = df_master['NX_per_country'].shift(1)
df_master.dropna(inplace=True)



In [8]:
df_master['Tag'] = pd.to_datetime(df_master['Tag'])

df_master = df_master.merge(df_prev_day_info,on='Tag')

In [9]:
df_master.head()

,Tag,NX_per_country,NX_per_country_prev_day,prev_day_consumption_ger,prev_day_price_deutschland/luxemburg,prev_day_price_dänemark 1,prev_day_price_dänemark 2,prev_day_price_frankreich,prev_day_price_italien (nord),prev_day_price_niederlande,...,3,4,5,6,7,8,9,10,11,12
0,2015-06-03,"[[5107.0, 3482.0, -653.0, 34.0, 446.0, 0.0, 1....","[[680.0, 3068.0, -894.0, -1090.0, -341.0, 0.0,...",1365352.75,44.390017,16.130417,16.7625,24.302083,43.613333,33.039583,...,0,0,0,1,0,0,0,0,0,0
1,2015-06-03,"[[5107.0, 3482.0, -653.0, 34.0, 446.0, 0.0, 1....","[[680.0, 3068.0, -894.0, -1090.0, -341.0, 0.0,...",1365352.75,44.390017,16.130417,16.7625,24.302083,43.613333,33.039583,...,0,0,0,1,0,0,0,0,0,0
2,2015-06-03,"[[5107.0, 3482.0, -653.0, 34.0, 446.0, 0.0, 1....","[[680.0, 3068.0, -894.0, -1090.0, -341.0, 0.0,...",1365352.75,44.390017,16.130417,16.7625,24.302083,43.613333,33.039583,...,0,0,0,1,0,0,0,0,0,0
3,2015-06-03,"[[5107.0, 3482.0, -653.0, 34.0, 446.0, 0.0, 1....","[[680.0, 3068.0, -894.0, -1090.0, -341.0, 0.0,...",1365352.75,44.390017,16.130417,16.7625,24.302083,43.613333,33.039583,...,0,0,0,1,0,0,0,0,0,0
4,2015-06-03,"[[5107.0, 3482.0, -653.0, 34.0, 446.0, 0.0, 1....","[[680.0, 3068.0, -894.0, -1090.0, -341.0, 0.0,...",1365352.75,44.390017,16.130417,16.7625,24.302083,43.613333,33.039583,...,0,0,0,1,0,0,0,0,0,0


---

# Models

In [10]:
def create_mlp(dim,regress=False):
    # define our MLP network
    model = Sequential()
    model.add(Dense(32, input_dim=dim, activation="relu"))
    model.add(Dense(64, activation="relu"))

    # check to see if the regression node should be added
    if regress:
        model.add(Dense(1, activation="linear"))
    return model

In [11]:
def create_cnn(width, height, depth=1,filters=(16, 32, 64),regress=False):
    # initialize the input shape and channel dimension, assuming
    # TensorFlow/channels-last ordering
    inputShape = (height, width, depth)

    # define the model input
    inputs = Input(shape=inputShape)

    x = inputs
    
    x = Conv2D(16,kernel_size=(4,1),strides=1, padding="same")(x)
    x = Activation("relu")(x)
    
    x = Dropout(0.2)(x)

    x = Conv2D(32,kernel_size=(1,width),strides=1, padding="same")(x)
    x = Activation("relu")(x)
    
    
    # flatten the volume, then FC => RELU => BN => DROPOUT
    x = Flatten()(x)
    x = Dense(height*width*2)(x)
    x = Activation("relu")(x)
    # construct the CNN
    model = Model(inputs, x)

    # return the CNN
    return model

# Training

In [13]:
# Prepare Data 
df_cnn = df_master[['Tag','NX_per_country_prev_day']]
df_mlp = df_master.drop('NX_per_country_prev_day',axis=1)

split = train_test_split(df_mlp,df_cnn, test_size=0.25, random_state=42)
(trainAttrX, testAttrX, trainImagesX, testImagesX) = split


testY = testAttrX[['NX_per_country']]
trainY = trainAttrX[['NX_per_country']]

trainAttrX.drop(['Tag','NX_per_country'],axis=1,inplace=True)
testAttrX.drop(['Tag','NX_per_country'],axis=1,inplace=True)


In [14]:
mlp = create_mlp(trainAttrX.shape[1])
cnn = create_cnn(10, 24,1)

In [15]:
# Combine Nets

combinedInput = concatenate([mlp.output, cnn.output])

x = Dense(24*10,kernel_initializer='normal')(combinedInput)
x = Reshape((24, 10))(x)
 
# our final model will accept categorical/numerical data on the MLP
# input and images on the CNN input, outputting a single value (the
# predicted price of the house)
model = Model(inputs=[mlp.input, cnn.input], outputs=x)
opt = Adam(lr=1e-3, decay=1e-3 / 200)
model.compile(loss="mean_absolute_error", optimizer=opt)

---

In [16]:
# Reshape
trainY = np.array(trainY['NX_per_country'].tolist())
testY = np.array(testY['NX_per_country'].tolist())

trainImagesX = np.expand_dims(np.array(trainImagesX['NX_per_country_prev_day'].tolist()), axis=3)
testImagesX = np.expand_dims(np.array(testImagesX['NX_per_country_prev_day'].tolist()), axis=3)

In [18]:
# train the model
print("[INFO] training model...")
model.fit(
    [trainAttrX, trainImagesX], trainY,
    validation_data=([testAttrX, testImagesX], testY),
    epochs=100, batch_size=10)

[INFO] training model...
Train on 26280 samples, validate on 8760 samples
Epoch 1/100
26280/26280 [==============================] - 182s 7ms/step - loss: 606.8155 - val_loss: 531.3986
Epoch 2/100
26280/26280 [==============================] - 183s 7ms/step - loss: 456.3679 - val_loss: 386.7923
Epoch 3/100
26280/26280 [==============================] - 208s 8ms/step - loss: 369.8594 - val_loss: 323.7356
Epoch 4/100
26280/26280 [==============================] - 225s 9ms/step - loss: 315.0421 - val_loss: 303.6883
Epoch 5/100
26280/26280 [==============================] - 234s 9ms/step - loss: 281.7729 - val_loss: 272.2760
Epoch 6/100
26280/26280 [==============================] - 213s 8ms/step - loss: 271.1008 - val_loss: 264.9025
Epoch 7/100
26280/26280 [==============================] - 197s 8ms/step - loss: 254.9510 - val_loss: 225.2892
Epoch 8/100
26280/26280 [==============================] - 210s 8ms/step - loss: 237.0795 - val_loss: 242.2735
Epoch 9/100
26280/26280 [=============

26280/26280 [==============================] - 191s 7ms/step - loss: 141.4413 - val_loss: 208.4592
Epoch 74/100
26280/26280 [==============================] - 193s 7ms/step - loss: 136.5471 - val_loss: 210.5477
Epoch 75/100
26280/26280 [==============================] - 193s 7ms/step - loss: 133.2146 - val_loss: 221.3192
Epoch 76/100
26280/26280 [==============================] - 193s 7ms/step - loss: 132.2049 - val_loss: 209.6082
Epoch 77/100
26280/26280 [==============================] - 193s 7ms/step - loss: 132.0366 - val_loss: 222.9106
Epoch 78/100
26280/26280 [==============================] - 194s 7ms/step - loss: 130.8483 - val_loss: 214.7846
Epoch 79/100
26280/26280 [==============================] - 193s 7ms/step - loss: 130.5601 - val_loss: 241.4036
Epoch 80/100
26280/26280 [==============================] - 192s 7ms/step - loss: 138.5045 - val_loss: 215.6285
Epoch 81/100
26280/26280 [==============================] - 194s 7ms/step - loss: 132.3224 - val_loss: 217.6356
Epoch

In [19]:
model.save("model/parallel_mlp_cnn_without_normalization.hdf5")

---

# Predict

In [20]:
preds = model.predict([testAttrX, testImagesX])

In [21]:
df_predictions = pd.DataFrame(columns=df.columns[2:12])
df_true_test_values = pd.DataFrame(columns=df.columns[2:12])

### Predictions

In [22]:
test_index = split[3]['Tag'].reset_index(drop=True)
ind = 0 


for pred in preds:
    data = pd.DataFrame(data=pred,columns=df.columns[2:12])
    data['Tag'] = test_index[ind] + np.arange(24) * datetime.timedelta(hours=1)

    if ind==0:
        df_predictions = data
    
    df_predictions = df_predictions.append(data)
    
    ind+=1

In [23]:
#df_predictions.iloc[:,0:9] = (df_predictions.iloc[:,0:9]*std_columns) + mean_columns

### Ground Truth

In [24]:
test_index = split[3]['Tag'].reset_index(drop=True)
ind = 0 


for test in testY:
    
    data = pd.DataFrame(data=test,columns=df.columns[2:12])
    
    data['Tag'] = test_index[ind] + np.arange(24) * datetime.timedelta(hours=1)

    if ind==0:
        df_true_test_values = data
    
    df_true_test_values = df_true_test_values.append(data)
    
    ind+=1

In [25]:
#df_true_test_values.iloc[:,0:9] = (df_true_test_values.iloc[:,0:9]*std_columns) + mean_columns

## Compare Results

In [27]:
((df_true_test_values.iloc[:,0:9] - df_predictions.iloc[:,0:9])**2).mean().mean()

50666433.339347675

In [35]:
df_predictions.loc[:,'NL':'PL'].iloc[0].sum()

10071.649

In [36]:
df_predictions.loc[:,'NX'].iloc[0]

10148.667